In [6]:
# Функции подается название трех файлов: sell.csv, supply.csv, inventory.csv

In [7]:
import pandas as pd
import csv


def store_inf(sell_1, supply_1, inventory_1):
    # считывание файлов sell.csv, supply.csv, inventory.csv
    # для одного магазина
    psell = pd.read_csv(sell_1, delimiter=',')
    psupply = pd.read_csv(supply_1, delimiter=',')
    pinventory = pd.read_csv(inventory_1, delimiter=',')

    # создаем два файла(берем первые символы названия из поданных файлов)
    # создаем файлы путем прибавления суффиксов steal.csv, daily.csv
    # steal.csv - количество сворованного товара за месяц
    # daily.csv - состояние склада в конце каждого дня
    file_name = sell_1[:6] + 'steal.csv'
    output_file = open(file_name, "w")
    file_name = sell_1[:6] + 'daily.csv'
    output_file1 = open(file_name, "w")
    wrtr = csv.writer(output_file)
    wrtr1 = csv.writer(output_file1)

    # Делаем заголовок в этих файлах
    wrtr.writerow(['date', 'apple', 'pen'])
    wrtr1.writerow(['date', 'apple', 'pen'])

    # Группируем данные по столбцу дата
    gb_1 = psell.groupby(['date'])['sku_num']

    # Переменные для обработки данных
    pen = 0
    apple = 0
    index_sup = 0
    index_inv = 0
    ap_new = 0
    pe_new = 0

    # Цикл по сгруппированным данным
    for date in gb_1:
        # Подсчет ежедневных продаж
        for i in date[1]:
            if i[6:8] == 'pe':
                pen += 1
            else:
                if i[6:8] == 'ap':
                    apple += 1

        # Учитываем, что 1 и 15 числа каждого месяца происходит поставка товара
        day = int(date[0].split('-')[2])
        if day == 1 or day == 15:
            ap_new = ap_new + psupply['apple'][index_sup]
            pe_new = pe_new + psupply['pen'][index_sup]
            index_sup += 1
        ap_new -= apple
        pe_new -= pen

        # На основе данных о состоянии склада в конце месяца, находим
        # сколько товара было украдено за месяц
        if date[0] == pinventory['date'][index_inv]:
            ap_steal = ap_new - pinventory['apple'][index_inv]
            pe_steal = pe_new - pinventory['pen'][index_inv]
            ap_new = pinventory['apple'][index_inv]
            pe_new = pinventory['pen'][index_inv]
            index_inv += 1
            wrtr.writerow([date[0], ap_steal, pe_steal])
        wrtr1.writerow([date[0], ap_new, pe_new])
        pen = 0
        apple = 0

    output_file.close()
    output_file1.close()

    # Создаем сводную таблицу о количестве проданного товара и украденного
    # по годам. Название путем добавления sold_stolen.csv
    file_name = sell_1[:6] + 'sold_stolen.csv'
    output_file2 = open(file_name, "w")
    wrtr2 = csv.writer(output_file2)
    wrtr2.writerow(['year', 'state', 'apple_sold', 'apple_stolen',
                    'pen_sold', 'pen_stolen'])

    # Считываем файл с уркаденным товаром
    f = pd.read_csv(sell_1[:6] + 'steal.csv', delimiter=',',
                    parse_dates=['date'], index_col='date')

    # Группируем файл по годам
    key = lambda x: x.year
    gb_1 = f.groupby(key)

    # Цикл для суммирования украденного товара по годам и запись в таблицу
    # В столбцах с проданным товаром пока ставим 0
    for year in gb_1:
        wrtr2.writerow([year[0], sell_1[:2], 0, year[1].sum()[0],
                        0, year[1].sum()[1]])
    output_file2.close()

    # Считываем данные с проданным товаром
    f = pd.read_csv(sell_1[:6] + 'sell.csv', delimiter=',',
                    parse_dates=['date'], index_col='date')
    f = f.sku_num
    key = lambda x: x.year
    gb_1 = f.groupby(key)
    index = 0

    # Изменяем файл sold_stolen.csv
    # Записываем число проданного товара по годам в таблицу
    tmp = pd.read_csv(sell_1[:6] + 'sold_stolen.csv', delimiter=',')
    for year in gb_1:
        apple = 0
        pen = 0
        for i in year[1]:
            if i[6:8] == 'pe':
                pen += 1
            else:
                if i[6:8] == 'ap':
                    apple += 1
        tmp.ix[index, 2] = apple
        tmp.ix[index, 4] = pen
        index += 1

        # Записываем изменения
    tmp.to_csv(sell_1[:6] + 'sold_stolen.csv', index=False)
    results_files.append(sell_1[:6] + 'sold_stolen.csv')
    return 0

In [8]:
# Считываем название файлов(по 3 файла), пока пользователь не введет "end"

In [9]:
end = 1
results_files = []
while end:
    # Чтение названий обрабатываемых файлов
    print("Enter the name of the sales file or 'end':")
    sell = input()
    if sell == "end":
        break
    print("Enter the name of the file supplies or 'end':")
    supply = input()
    if supply == "end":
        break
    print("Enter the name of the inventory file or 'end':")
    inventory = input()
    if inventory == "end":
        break
    store_inf(sell, supply, inventory)

Enter the name of the sales file or 'end':
MS-b1-sell.csv
Enter the name of the file supplies or 'end':
MS-b1-supply.csv
Enter the name of the inventory file or 'end':
MS-b1-inventory.csv
Enter the name of the sales file or 'end':
MS-b2-sell.csv
Enter the name of the file supplies or 'end':
MS-b2-supply.csv
Enter the name of the inventory file or 'end':
MS-b2-inventory.csv
Enter the name of the sales file or 'end':
MS-m1-sell.csv
Enter the name of the file supplies or 'end':
MS-m1-supply.csv
Enter the name of the inventory file or 'end':
MS-m1-inventory.csv
Enter the name of the sales file or 'end':
MS-m2-sell.csv
Enter the name of the file supplies or 'end':
MS-m2-supply.csv
Enter the name of the inventory file or 'end':
MS-m2-inventory.csv
Enter the name of the sales file or 'end':
MS-s1-sell.csv
Enter the name of the file supplies or 'end':
MS-s1-supply.csv
Enter the name of the inventory file or 'end':
MS-s1-inventory.csv
Enter the name of the sales file or 'end':
MS-s2-sell.csv
En

In [10]:
# Объединим все итоговые файлы
# Сортируем по году и штату
# Суммируем результаты строк, у которых совпадают год и штат
# Записываем результат в файл "all_sold_stolen.csv"

In [11]:
results_files_read = []
for i in results_files:
    results_files_read.append(pd.read_csv(i, delimiter=','))

result = pd.concat(results_files_read)

result = result.sort(['year', 'state'], ascending=[1, 1])

gb = result.groupby(['year', 'state']).sum()

gb.to_csv('all_sold_stolen.csv.csv', index=True)

/Users/violetta/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
